# 以下代码操作需要基于抓包软件 Fiddler Classic

https://www.telerik.com/download/fiddler

In [1]:
# 初始化 Jupyter Notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"
# %matplotlib inline
# from matplotlib import pyplot as plt
# plt.rcParams["font.sans-serif"] = ["SimHei"]

In [2]:
import emoji
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By

# 设置请求需要带上的 headers

In [4]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36 NetType/WIFI MicroMessenger/7.0.20.1781(0x6700143B) WindowsWechat(0x63060012)",
    "Cookie": "你的cookie"
}

# 打开微信电脑客户端
1. 开启 Fiddler Classic 抓包
1. 随便找一篇公众号推文点进去
1. 点开文章后查看 Fiddler Classic 的抓包 url (一般是蓝色)

In [70]:
# 这个 url 是微信公众文章的 url ,里面含有很多信息.
# 必须用微信客户端打开,只能用 Fiddler 抓取到.
# 并且,这个 url 的 `key` 和 `export_key` 关键字会定时变换,和下面的 `appmsg_token` 要记得更新
url = "https://mp.weixin.qq.com/s?__biz=MzIyMDAyMzg2NA==&mid=2247502017&idx=1&sn=cc0d77ed0b4ecc6d1ed028c496b7f088&chksm=97d0d596a0a75c80010af6e0c8186d216efb8776c295ee1f928f8cf0531be665459f48533f18&scene=20&xtrack=1&key=5115e9b6036e2f42eb2062e05c25949fc1e75962915c5fa2640e8b5adf16ad7cb5ffacd40057c031f95bc9f4d5df325bca2eb14c785c01ddaaf849d6df6fbc1db4c0d019026e91d26bf7e4edc4ac16e006a0c8fa8eda1a8c8f032cea2ba17d1cf2cd09f15de8331536492f1efe95a3e3f69146dea3f00ec54c1f16cd37cb405b&ascene=51&uin=NTMyMDg4NDIz&devicetype=Windows+10+x64&version=63060012&lang=zh_CN&session_us=gh_ede3aa4af2f7&exportkey=A27ub8i77llM2QuBmR4f580%3D&acctmode=0&pass_ticket=xZe7PPe%2FlNG6huc41mX8Lrd873Bm0WlC1CWbxp5Q1s0dm2it3PSEYfBGg974VFBr&wx_header=0&fontgear=2"
# 这个是"个人安全"意义的 token ,几十分钟内有效
appmsg_token = "1166_9KvxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxVwr--u0"

# 获取关键数据

In [ ]:
# 直接用 split 方法拿出url里的关键信息 `__biz`
__biz = url.split("&")[0].split("__biz=")[1]
__biz
# 将剩余的参数拿出来并做成一个字典 `parameters -> dict()`
parameters = dict([i.split("=") for i in url.split("&")[1:]])
parameters
# 访问时需要用到发出这个关键数据来获得响应
appmsgext_data = {
    "uin": parameters["uin"],
    "key": parameters["key"],
    "is_only_read": "1",
    # 下面这三个可有可无,先注释掉.
    # "pass_ticket": parameters["pass_ticket"],
    # "is_temp_url": "0",
    # "appmsg_type": "9"
}

###### 阅读量, 点赞量, 在看量 ######
# 阅读量, 点赞量, 在看量都在这个 **额外加载的** 页面上
appmsgext_base = "https://mp.weixin.qq.com/mp/getappmsgext?"
# 一个公众号一个 `__biz`, 一篇文章一个 `mid`, `sn`, 然后用 format 方法拼接url
# 这里的拼接只需要关键参数
# 何为关键？就是在浏览器删掉一个参数后无法看到数据的，就是关键的。
appmsgext_url = appmsgext_base + "__biz={}&mid={}&sn={}&idx={}&appmsg_token={}".format(__biz, parameters["mid"], parameters["sn"], parameters["idx"], appmsg_token)
# 注意是 requests.post
three = requests.post(appmsgext_url, headers=headers, data=appmsgext_data).json()
print("阅读量 %d\n点赞数 %d\n在看数 %d\n"%(three["appmsgstat"]["read_num"], three["appmsgstat"]["old_like_num"],three["appmsgstat"]["like_num"]))

###### 用户评论 ######
comment_base = "https://mp.weixin.qq.com/mp/appmsg_comment?action=getcomment"
# 同样的，也是只需要关键参数.这里面的 `comment_id` 也是一篇文章一个，需要换
comment_url = comment_base + "&appmsgid={}&comment_id={}&limit={}&uin={}&key={}&__biz={}".format(parameters["mid"],"2129903392402571268","100",parameters["uin"],parameters["key"],__biz)
comment_url
comment_data = {
    "offset":"0",
    "business_id":"0",
    "idx":"1",
    "sessionid":"svr_f4617534cf2",
    "enterid":"1653132238",
    "pass_ticket":"xZe7PPe/lNG6huc41mX8Lrd873Bm0WlC1CWbxp5Q1s0dm2it3PSEYfBGg974VFBr"
}
# 这里需要 requests.get
comment = requests.get(comment_url, headers=headers).json()
comment

# 下面两段代码只是为了展示评论

In [ ]:
for i in comment["elected_comment"]:
    print(i)

In [ ]:
comment_relationship = {
    "id": [],
    "content": [],
    "like_num": [],
    "reply_id": [],
    "reply_content": [],
}
for i in comment["elected_comment"]:
    comment_relationship["id"].append(str(i["id"]))
    comment_relationship["content"].append(i["content"])
    comment_relationship["like_num"].append(str(i["like_num"]))
    if i["reply"]["reply_list"] != []:
        comment_relationship["reply_id"].append(str(i["reply"]["reply_list"][0]["reply_id"]))
        comment_relationship["reply_content"].append(i["reply"]["reply_list"][0]["content"])
    else:
        comment_relationship["reply_id"].append(None)
        comment_relationship["reply_content"].append(None)
pd.DataFrame(comment_relationship)